In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
import pandas as pd

# 데이터 불러오기

### 각 카테고리별로 크롤링한 데이터 병합

In [ ]:
dy = [220, 310, 320, 400, 405, 410, 420, 430, 500]
ty = [600, 700, 750, 800, 810, 900]
mh = [910, 920, 930, 980, 990, 999]
df = pd.DataFrame()
for i in dy+ty+mh:
    temp = pd.read_csv(f"data/crawling/{i}.csv")
    if 'warning' in temp.columns:
        temp = temp.drop('warning', axis=1)
    if 'address_id' in temp.columns:
        temp = temp.drop('address_id', axis=1)
    df = df.append(temp)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


데이터 개수 확인

In [ ]:
len(df)

604716

컬럼 종류 확인

In [ ]:
df.columns

Index(['pid', 'uid', 'name', 'status', 'price', 'num_faved', 'num_item_view',
       'num_comment', 'user_name', 'update_time', 'free_shipping',
       'profile_image', 'product_image', 'bizseller', 'category_id',
       'category_name', 'tradable', 'used', 'used_code', 'image_count',
       'image_file_list_for_edit', 'image_source', 'location', 'latitude',
       'longitude', 'qty', 'keyword', 'contact_hope', 'bunpay',
       'need_induce_bun_pay_filter', 'is_location_confirm',
       'only_neighborhood', 'neighborhood_option', 'pay_option',
       'restriction_status', 'is_ad', 'is_adult', 'style', 'group_ids',
       'groups', 'is_buncar', 'checkout', 'naver_shopping_url',
       'extended_spec', 'ordernow_label', 'ordernow_token_required',
       'contact_enabled', 'comment_enabled', 'is_free_sharing', 'num_item',
       'num_grade_avg', 'num_follower', 'num_review', 'is_withdraw',
       'is_blocked', 'join_date', 'access_time', 'is_identification',
       'badges'],
      dtype=

## 전처리

### 기존 데이터 형식에 맞게 이름 변경

In [ ]:
rename_dict = {'pid': 'content_id', 
               'uid': 'adv_id', 
               'name': 'content_name',
               'status': 'content_status',
               'price': 'content_price',
               'num_faved': 'content_likes',
               'num_item_view': 'content_views',
               'num_comment': 'content_comment_count', 
               'user_name': 'adv_name',
               'free_shipping': 'content_delivery_fee',
               'used_code': 'content_used',
               'location': 'content_place',
               'need_induce_bun_pay_filter': 'content_b_pay',
               'num_item': 'adv_item_count',
               'num_grade_avg': 'adv_grade',
               'num_follower': 'adv_follower_count',
               'num_review': 'adv_review_count',
           }


### drop할 컬럼 목록 정리

In [ ]:
drop_list = ['contact_enabled', 'naver_shopping_url', 'bizseller', 'groups', 'is_withdraw',
            'category_id', 'ordernow_token_required', 'only_neighborhood', 'is_identification',
            'is_adult', 'style', 'group_ids', 'ordernow_label', 'checkout', 'is_free_sharing',
            'is_blocked', 'extended_spec', 'contact_hope', 'used', 'comment_enabled',
            'restriction_status', 'is_buncar', 'badges', 'category_name', 'product_image', 'pay_option']

### 장소 데이터를 시/도 형식으로 변환

- 결측치는 `전국`으로 처리
- 아래 딕셔너리로 변환했을 때 변환이 안되는 값들이 전체 60만개 중 1000개정도 존재  
    -> `전국`으로 처리

In [ ]:
place_dict = {
    '서울': '서울특별시',
    '강남': '서울특별시',
    '잠실': '서울특별시',
    '경기도': '경기도', '경기': '경기도',
    '강원': '강원도', '제주': '제주도',
    '경상북': '경상북도', '경북': '경상북도',
    '경상남': '경상남도', '경남': '경상남도',
    '전라북': '전라북도', '전북': '전라북도',
    '전라남': '전라남도', '전남': '전라남도',
    '충청북': '충청북도', '충북': '충청북도',
    '충청남': '충청남도', '충남': '충청남도',
    '인천': '인천광역시',
    '대구': '대구광역시',
    '대전': '대전광역시',
    '부산': '부산광역시',
    '울산': '울산광역시',
    '광주': '광주광역시',
    '세종': '세종시'
}

def sido_new(place):
    if isinstance(place, str):
        for name in place_dict.keys():
            if name in place:
                place = place_dict[name]
            else:
                palce = '전국'
    else:
        place = '전국'
    return place

### 컬럼 순서 보기 좋게 정렬

In [ ]:
sort_col = ['content_id', 'content_name', 'adv_id', 'adv_name', 'keyword', 'content_status',
       'content_price', 'content_used', 'content_likes', 'content_views',
       'content_comment_count', 'content_delivery_fee',
       'content_cat_1', 'content_cat_2', 'content_cat_3',
       'content_b_pay', 
       'adv_item_count', 'adv_grade', 'adv_follower_count', 'adv_review_count',
       'tradable', 'qty', 
       'content_place', 'latitude', 'longitude', 'is_location_confirm', 
       'image_count', 'image_file_list_for_edit', 'image_source', 'profile_image',
       'neighborhood_option', 'is_ad', 'bunpay', 'pay_option_in_person', 'pay_option_bun_pay_filter_enabled', 
       'update_time', 'access_time', 'join_date', 
       'seller_name']

### 위 과정을 모아서 전처리

In [ ]:
def preprocess_crawling(df_):
    df = df_.copy()
    df = df.rename(rename_dict, axis=1)
    df = df.reset_index().drop('index', axis=1)

    # 대분류
    df['content_cat_1'] = df['category_name'].apply(lambda x : x.split("'")[3])
    # 중분류
    df['content_cat_2'] = df['category_name'].apply(lambda x : x.split("'")[7] if len(x.split("'"))>6 else x.split("'")[3])
    # 소분류
    df['content_cat_3'] = df['category_name'].apply(lambda x : x.split("'")[-2])
    # delivery_fee -> train 데이터와 같게
    df['content_delivery_fee'] = df['content_delivery_fee'].apply(lambda x : 1*x)
    # b_pay -> train 데이터와 같게
    df['content_b_pay'] = df['content_b_pay'].apply(lambda x : 1 - 1*x)
    # 중고 여부를 train 데이터와 같게
    df['content_used'] = df['content_used'].apply(lambda x: 0 if x==2 else 1)
    # 지역명을 시/도 형식에 맞게
    df['content_place'] = df['content_place'].apply(sido_new)
    # pay option에서 필요한 부분만 사용
    df['pay_option_in_person'] = df['pay_option'].apply(lambda x : x.split("'")[4][2:-2])
    df['pay_option_bun_pay_filter_enabled'] = df['pay_option'].apply(lambda x : x.split("'")[6][2:-2])
    # unix time을 실제 시간으로
    df['update_time'] = pd.to_datetime(df['update_time'], unit='s')
    df['access_time'] = pd.to_datetime(df['access_time'], unit='s')
    df['join_date'] = pd.to_datetime(df['join_date'], unit='s')
    
    df['seller_name'] = df['badges'].apply(lambda x : x[94:97])

    # 필요없는 컬럼 drop
    df = df.drop(drop_list, axis=1)
    # 보기 좋게 정렬
    df = df[sort_col]
    return df

위에서 병합한 데이터 전처리 진행

In [ ]:
df = preprocess_crawling(df)

# CTR 예측 모델 학습(기존 데이터 사용)

## 학습 데이터 준비

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
train_data = pd.read_csv('data/train.csv')

### 범주형/연속형 feature 구분

In [ ]:
categorical = [
    'viewer_gender',  
    'content_used',
    'content_cat_1',
    'content_delivery_fee',
    'content_b_pay'
    ]
    
continuous = [
    'bid_price', 
    'content_price',
    'content_comment_count',     
    'content_views',       
    'content_likes',
    'adv_item_count',
    'adv_follower_count', 
    'adv_review_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    "viewer_parcel_post_count",
    'viewer_transfer_count',
    'viewer_chat_count', 
    ]

features = categorical + continuous

### 전처리

In [ ]:
# 수동 라벨인코딩
cat_label = {}
for i, cat in enumerate(df_total['content_cat_1'].unique()):
    cat_label[cat] = i
train_data['content_cat_1'] = train_data['content_cat_1'].apply(lambda x: cat_label[x])
train_data['content_delivery_fee'] = train_data['content_delivery_fee'].apply(lambda x: 1 if x=='배송비포함' else 0)
# 스케일링, scaler 남겨두기
scaler = StandardScaler()
train_data[continuous] = scaler.fit_transform(train_data[continuous])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data['label'], test_size=0.2, random_state=47)

### 평가지표 정의

In [ ]:
average_ctr = df_total['label'].mean()

# 평가지표 함수

def get_rig(train_y, test_y, pred, avg_ctr):
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig

def evaluate(pred):
    print("")
    print("test LogLoss", round(log_loss(y_test, pred), 4))
    print("test AUC", round(roc_auc_score(y_test, pred), 4))
    print("test RIG", round(get_rig(y_train, y_test, pred, average_ctr), 4))

## LGB 학습

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.LGBMClassifier(n_estimators=200, 
                           random_state=47, 
                           learning_rate=0.1,
                           boosting_type='gbdt', 
                           num_leaves=255, 
                           max_depth=20, 
                           min_data_in_leaf=200, 
                           zero_as_missing=True,
                           objective='binary')
model_lgb.fit(X_train, y_train)

In [ ]:
pred_lgb = model.predict_proba(X_test)[:,1]
evaluate(pred_lgb)

### Finalize

In [ ]:
model_lgb = lgb.LGBMClassifier(n_estimators=200, 
                           random_state=47, 
                           learning_rate=0.1,
                           boosting_type='gbdt', 
                           num_leaves=255, 
                           max_depth=20, 
                           min_data_in_leaf=200, 
                           zero_as_missing=True,
                           objective='binary')

X_final = X_train.append(X_test)
y_final = y_train.append(y_test)

model_lgb.fit(X_final, y_final)


test LogLoss 0.1271
test AUC 0.7675
test RIG 0.102


### 모델 저장

In [ ]:
# import joblib

# save model
# joblib.dump(model_lgb, '경로/파일명.pkl')
 
# # load model
# load_model = joblib.load('김동영/lgb.pkl')

['lgb_ex.pkl']

## DeepFM 학습

In [ ]:
import torch
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, X_train[feat].nunique())
                        for feat in categorical] + [DenseFeat(feat, 1, )
                                                        for feat in continuous]

# dnn feature에 일단 모든 feature 정보 넣기
dnn_feature_columns = fixlen_feature_columns

# lenear feature에도 일단 모든 feature 정보 넣기
linear_feature_columns = fixlen_feature_columns 

# feature 이름들만 따로 저장
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# train/test 데이터의 각 feature에 대응하는 컬럼을 딕셔너리로 저장
train_model_input = {name: X_train[name] for name in feature_names}
test_model_input = {name: X_test[name] for name in feature_names}


In [ ]:
device = 'cpu'
use_gpu = True
if use_gpu and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
model = DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
           dnn_activation='relu', dnn_use_bn=True, task='binary', device=device)

# optimizer, loss 설정
model.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"], )

# 모델 학습
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=2, patience=5, mode='auto')
hist = model.fit(x = train_model_input, y = y_train.values , batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

pred_dfm = model.predict(test_model_input, 512)
evaluate(pred_dfm)

cuda ready...
cuda:0
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
14s - loss:  0.1419 - binary_crossentropy:  0.1419 - auc:  0.6591 - val_binary_crossentropy:  0.1343 - val_auc:  0.6629
Epoch 2/50
14s - loss:  0.1367 - binary_crossentropy:  0.1367 - auc:  0.6740 - val_binary_crossentropy:  0.1328 - val_auc:  0.6824
Epoch 3/50
9s - loss:  0.1355 - binary_crossentropy:  0.1355 - auc:  0.6883 - val_binary_crossentropy:  0.1333 - val_auc:  0.6810
Epoch 4/50
10s - loss:  0.1352 - binary_crossentropy:  0.1352 - auc:  0.6909 - val_binary_crossentropy:  0.1326 - val_auc:  0.6842
Epoch 5/50
12s - loss:  0.1347 - binary_crossentropy:  0.1347 - auc:  0.6959 - val_binary_crossentropy:  0.1321 - val_auc:  0.6958
Epoch 6/50
9s - loss:  0.1346 - binary_crossentropy:  0.1345 - auc:  0.6982 - val_binary_crossentropy:  0.1311 - val_auc:  0.7013
Epoch 7/50
10s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.7038 - val_binary_crossentropy:  0.1322 - val_auc:

### Finalize

In [ ]:
device = 'cpu'
use_gpu = True
if use_gpu and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

X_final = X_train.append(X_test)
y_final = y_train.append(y_test)
final_model_input = {name: X_final[name] for name in feature_names}


# 모델 정의
model_DFM = DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
           dnn_activation='relu', dnn_use_bn=True, task='binary', device=device)

# optimizer, loss 설정
model_DFM.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"], )

# 모델 학습
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=2, patience=5, mode='auto')
hist = model_DFM.fit(x = final_model_input, y = y_final.values , batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

cuda ready...
cuda:0
Train on 588555 samples, validate on 147139 samples, 1150 steps per epoch
Epoch 1/50
14s - loss:  0.1406 - binary_crossentropy:  0.1406 - auc:  0.6584 - val_binary_crossentropy:  0.1365 - val_auc:  0.6750
Epoch 2/50
12s - loss:  0.1356 - binary_crossentropy:  0.1356 - auc:  0.6800 - val_binary_crossentropy:  0.1350 - val_auc:  0.6924
Epoch 3/50
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6900 - val_binary_crossentropy:  0.1341 - val_auc:  0.6984
Epoch 4/50
12s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.6954 - val_binary_crossentropy:  0.1344 - val_auc:  0.6945
Epoch 5/50
17s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.6988 - val_binary_crossentropy:  0.1345 - val_auc:  0.6995
Epoch 6/50
14s - loss:  0.1335 - binary_crossentropy:  0.1335 - auc:  0.7032 - val_binary_crossentropy:  0.1342 - val_auc:  0.7034
Epoch 7/50
21s - loss:  0.1332 - binary_crossentropy:  0.1332 - auc:  0.7055 - val_binary_crossentropy:  0.1336 - val_a

### 모델 저장

In [ ]:
# import torch
# model = DeepFM()
# torch.save(model_DFM, 'DeepFM.h5')
# model_DFM = torch.load('DeepFM.h5')

# CTR 예측

### 가상 유저군 생성

In [ ]:
from itertools import product

In [ ]:
# # 기본

grids = {'bid_price': [50.0],
         'viewer_gender': [0, 1, 2],
         'viewer_age': [20, 30]
         }

# 신규 유저 정보
new_user = {
        'viewer_chat_count': 0.0,
        'viewer_following_count': 0.0,
        'viewer_parcel_post_count': 0.0,
        'viewer_pay_count': 0.0,
        'viewer_transfer_count': 0.0}

# 기존 유저 정보
old_user = {
        'viewer_chat_count': 10.0,
        'viewer_following_count': 5.0,
        'viewer_parcel_post_count': 5.0,
        'viewer_pay_count': 5.0,
        'viewer_transfer_count': 5.0}

user_list = []
for i, values in enumerate(list(product(*grids.values()))):
    point = dict(zip(grids.keys(), values))
    # 신규 유저
    grid_params_new = {**new_user, **point}
    user_list.append(grid_params_new)
    # 기존 유저
    grid_params_old = {**old_user, **point}
    user_list.append(grid_params_old)

user_list

[{'bid_price': 50.0,
  'viewer_age': 20,
  'viewer_chat_count': 0.0,
  'viewer_following_count': 0.0,
  'viewer_gender': 0,
  'viewer_parcel_post_count': 0.0,
  'viewer_pay_count': 0.0,
  'viewer_transfer_count': 0.0},
 {'bid_price': 50.0,
  'viewer_age': 20,
  'viewer_chat_count': 10.0,
  'viewer_following_count': 5.0,
  'viewer_gender': 0,
  'viewer_parcel_post_count': 5.0,
  'viewer_pay_count': 5.0,
  'viewer_transfer_count': 5.0},
 {'bid_price': 50.0,
  'viewer_age': 30,
  'viewer_chat_count': 0.0,
  'viewer_following_count': 0.0,
  'viewer_gender': 0,
  'viewer_parcel_post_count': 0.0,
  'viewer_pay_count': 0.0,
  'viewer_transfer_count': 0.0},
 {'bid_price': 50.0,
  'viewer_age': 30,
  'viewer_chat_count': 10.0,
  'viewer_following_count': 5.0,
  'viewer_gender': 0,
  'viewer_parcel_post_count': 5.0,
  'viewer_pay_count': 5.0,
  'viewer_transfer_count': 5.0},
 {'bid_price': 50.0,
  'viewer_age': 20,
  'viewer_chat_count': 0.0,
  'viewer_following_count': 0.0,
  'viewer_gender': 1

In [ ]:
df

,content_id,content_name,adv_id,adv_name,keyword,content_status,content_price,content_used,content_likes,content_views,content_comment_count,content_delivery_fee,content_cat_1,content_cat_2,content_cat_3,content_b_pay,adv_item_count,adv_grade,adv_follower_count,adv_review_count,tradable,qty,content_place,latitude,longitude,is_location_confirm,image_count,image_file_list_for_edit,image_source,profile_image,neighborhood_option,is_ad,bunpay,pay_option_in_person,pay_option_bun_pay_filter_enabled,update_time,access_time,join_date,seller_name,lgb_20세_?_신규,dfm_20세_?_신규,lgb_20세_?_기존,dfm_20세_?_기존,lgb_30세_?_신규,dfm_30세_?_신규,lgb_30세_?_기존,dfm_30세_?_기존,lgb_20세_남성_신규,dfm_20세_남성_신규,lgb_20세_남성_기존,dfm_20세_남성_기존,lgb_30세_남성_신규,dfm_30세_남성_신규,lgb_30세_남성_기존,dfm_30세_남성_기존,lgb_20세_여성_신규,dfm_20세_여성_신규,lgb_20세_여성_기존,dfm_20세_여성_기존,lgb_30세_여성_신규,dfm_30세_여성_신규,lgb_30세_여성_기존,dfm_30세_여성_기존
287746,174290766,디즈니 방풍 롱 니트 집업,4871683,10cm상점,"디즈니방풍니트,파리게이츠니트,까스텔바작방풍,먼싱방풍,여성골프니트",0,18000,1,1,29,0,0,스포츠/레저,골프,골프 여성 의류,0,5706,10,739,136,False,1,대구광역시,35.814767,128.524213,1,4,[],NaN,https://media.bunjang.co.kr/images/crop/715531...,display_on_neighborhood,False,True,True,False,2021-12-29 14:29:40,2021-11-28 06:01:24,2016-09-09 12:45:28,*만*,0.009660,0.342921,0.004850,0.109352,0.009660,0.325444,0.004850,0.080333,0.003042,0.338875,0.002513,0.128139,0.003042,0.397728,0.002513,0.114332,0.008399,0.155748,0.008594,0.055941,0.008399,0.180871,0.008594,0.042714
288003,174527003,엠유스포츠 방풍 울 니트 롱 니트 원피스,4871683,10cm상점,"파리게이츠,엠유스포츠,먼싱웨어,타이틀리스트,까스텔바작",0,29000,1,0,29,0,0,스포츠/레저,골프,골프 여성 의류,0,5706,10,739,136,False,1,대구광역시,35.814767,128.524213,1,4,[],NaN,https://media.bunjang.co.kr/images/crop/715531...,display_on_neighborhood,False,True,True,False,2021-12-29 14:13:53,2021-11-28 06:01:24,2016-09-09 12:45:28,*만*,0.009660,0.332961,0.004850,0.108371,0.009660,0.315436,0.004850,0.079758,0.003042,0.323959,0.002513,0.127534,0.003042,0.383300,0.002513,0.113863,0.008399,0.149407,0.008594,0.055378,0.008399,0.174243,0.008594,0.042971
287747,174290969,나이키 테크팩 기능성 퀼팅 집업,4871683,10cm상점,"나이키테크팩집업,나이키스우시집업,나이키골프,여성골프웨어,파리게이츠",0,39000,1,6,85,0,0,스포츠/레저,골프,골프 여성 의류,0,5706,10,739,136,False,1,대구광역시,35.814767,128.524213,1,4,[],NaN,https://media.bunjang.co.kr/images/crop/715531...,display_on_neighborhood,False,True,True,False,2021-12-29 14:29:40,2021-11-28 06:01:24,2016-09-09 12:45:28,*만*,0.009660,0.335800,0.004850,0.109059,0.009660,0.317801,0.004850,0.080490,0.003042,0.322046,0.002513,0.127941,0.003042,0.381018,0.002513,0.114218,0.008399,0.151655,0.008594,0.055257,0.008399,0.176311,0.008594,0.043379
293959,174491852,J-8938 코오롱스포츠 여성용 고어윈드스토퍼 후드집업 90,75393972,2021년화이팅,코오롱스포츠,0,20000,1,1,16,0,0,스포츠/레저,등산/클라이밍,여성 등산복,0,5573,10,54,27,False,1,전국,NaN,NaN,0,5,[],NaN,NaN,not_verified,False,True,True,False,2021-12-29 08:42:05,2021-12-28 23:30:19,2021-02-10 07:20:35,*성*,0.009660,0.344007,0.004850,0.099129,0.009660,0.279458,0.004850,0.068552,0.003042,0.297237,0.002513,0.110835,0.003042,0.376802,0.002513,0.098800,0.008399,0.154834,0.008594,0.053349,0.008399,0.164799,0.008594,0.037578
396140,147505386,(원가띵 마테컷) 당근곰/ 뽐뽐스튜디오 / 람찌네소품샵,970589,떡메랩핑지띵,"마테컷,커벨,람찌네,제니빌리지,무심한하루",0,470,1,157,6645,0,0,도서/티켓/문구,문구,학습도구/문구/필기류,1,12,10,485,1065,False,13,전국,NaN,NaN,0,4,[],ALBUM,https://media.bunjang.co.kr/images/crop/429921...,not_verified,False,True,True,False,2021-12-30 05:10:45,2021-12-29 01:01:15,2013-05-16 13:51:05,*윤*,0.006022,0.518553,0.012931,0.170921,0.006022,0.535198,0.012931,0.138372,0.002132,0.371852,0.008028,0.143527,0.002132,0.366362,0.008028,0.123422,0.005089,0.188870,0.015652,0.067707,0.005089,0.203812,0.015652,0.052055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554436,174534891,동전매입합니다,77390952,아무나사가게욕,"희귀동전,옛날동전,100원,에어팟프로,버즈프로",0,777777777,1,0,3,0,0,예술/희귀/수집품,희귀/수집품,희귀/수집품,0,0,0,0,0,False,1,전국,NaN,NaN,0,1,

### 위 유저들에 대해서 예측

In [ ]:
# 크롤링해서 얻은 게시글, 광고주 정보
candidates = ['adv_follower_count',
                  'adv_item_count',
                  'adv_review_count',
                  'content_b_pay',
                  'content_cat_1',
                  'content_comment_count',
                  'content_delivery_fee',
                  'content_likes',
                  'content_price',
                  'content_used',
                  'content_views']

# 위에서 생성한 유저 목록에서
for demo_user in user_list:
    candidates_df = df[candidates].copy()
    # 각 유저마다 특성을 입력(나이, 성별 등)
    for feat in demo_user.keys():
        candidates_df[feat] = demo_user[feat]
    
    # 전처리 진행
    candidates_df['content_cat_1'] = candidates_df['content_cat_1'].apply(lambda x: cat_label[x])
    # 학습 시 사용했던 sclaer 사용
    candidates_df[continuous] = scaler.transform(candidates_df[continuous])
    # DeepFM input 형태에 맞게
    pred_model_input = {name: candidates_df[name] for name in candidates_df.columns}

    # 각 모델별 CTR 예측
    CTR_lgb = model_lgb.predict_proba(candidates_df)[:,1]
    CTR_dfm = model_DFM.predict(pred_model_input, 512)
    
    # 해당 유저의 정보
    user_age = demo_user['viewer_age']
    user_gender = ['?', '남성', '여성'][demo_user['viewer_gender']]
    user_new = ['신규', '기존'][int(demo_user['viewer_chat_count']/10)]

    # 각 유저에 대해 모델별 클릭율 저장
    df[f'lgb_{user_age}세_{user_gender}_{user_new}'] = CTR_lgb
    df[f'dfm_{user_age}세_{user_gender}_{user_new}'] = CTR_dfm

    # 모델별 평균 클릭율 출력
    print(f"유저 - 나이: {user_age}, 성별: {user_gender}, {user_new}")
    print(f"클릭율 - LightGBM : {round(100*CTR_lgb.mean(),2)}%, deepFM: {round(100*CTR_dfm.mean(),2)}%, 학습평균 : {round(100*average_ctr, 2)}%")

유저 - 나이: 20, 성별: ?, 신규
클릭율 - LightGBM : 2.91%, deepFM: 2.81%, 학습평균 : 3.2
유저 - 나이: 20, 성별: ?, 기존
클릭율 - LightGBM : 1.88%, deepFM: 2.41%, 학습평균 : 3.2
유저 - 나이: 30, 성별: ?, 신규
클릭율 - LightGBM : 2.91%, deepFM: 2.95%, 학습평균 : 3.2
유저 - 나이: 30, 성별: ?, 기존
클릭율 - LightGBM : 1.88%, deepFM: 1.94%, 학습평균 : 3.2
유저 - 나이: 20, 성별: 남성, 신규
클릭율 - LightGBM : 1.41%, deepFM: 3.58%, 학습평균 : 3.2
유저 - 나이: 20, 성별: 남성, 기존
클릭율 - LightGBM : 1.03%, deepFM: 2.47%, 학습평균 : 3.2
유저 - 나이: 30, 성별: 남성, 신규
클릭율 - LightGBM : 1.41%, deepFM: 3.48%, 학습평균 : 3.2
유저 - 나이: 30, 성별: 남성, 기존
클릭율 - LightGBM : 1.03%, deepFM: 1.99%, 학습평균 : 3.2
유저 - 나이: 20, 성별: 여성, 신규
클릭율 - LightGBM : 3.89%, deepFM: 1.82%, 학습평균 : 3.2
유저 - 나이: 20, 성별: 여성, 기존
클릭율 - LightGBM : 3.01%, deepFM: 1.77%, 학습평균 : 3.2
유저 - 나이: 30, 성별: 여성, 신규
클릭율 - LightGBM : 3.89%, deepFM: 1.96%, 학습평균 : 3.2
유저 - 나이: 30, 성별: 여성, 기존
클릭율 - LightGBM : 3.01%, deepFM: 1.43%, 학습평균 : 3.2


## HTML로 시각화해보기

In [ ]:
from IPython.display import HTML, display

In [ ]:
# 기준 모델, 유저정보 입력
model_name = 'dfm'
user_age = 30
user_gender = '남성'
user_new = '신규'
ctr_name = f'{model_name}_{user_age}세_{user_gender}_{user_new}'
# 기준에 따라 정렬
df = df.sort_values(ctr_name, ascending=False)

# HTML로 시각화
html = ""
for idx, row in df[0:20].iterrows():
    html += f'''
        <div style="display:inline-block;min-width:150px;max-width:150px;vertical-align:top">
        <ul>
            <li>광고명: {row.content_name}</li>
            <li>가격: {row.content_price}</li>
            <li>카테고리: {row.content_cat_1}</li>
            <li>CTR: {round(100*row[ctr_name], 2)}%</li>
        </ul>
        <img src="https://media.bunjang.co.kr/product/{row.content_id}_..." style="width:150px;">
        </div>
        '''
    
display(HTML(html))